In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import shutil

# --- 1. CONFIGURATION ---
RAW_DATA_DIR = "../content/dataset"  # Input
PROCESSED_DATA_DIR = "../content/processed_data" # Output
CLASSES = ["Hemorrhagic", "Ischemic", "Tumor"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TARGET_SIZE = (224, 224)

# --- 2. RELOAD THE U-NET ---
class SimpleUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.e1 = nn.Sequential(nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(), nn.Conv2d(64, 64, 3, padding=1), nn.ReLU())
        self.pool = nn.MaxPool2d(2)
        self.e2 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.Conv2d(128, 128, 3, padding=1), nn.ReLU())
        self.b = nn.Sequential(nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(), nn.Conv2d(256, 256, 3, padding=1), nn.ReLU())
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.d2 = nn.Sequential(nn.Conv2d(256 + 128, 128, 3, padding=1), nn.ReLU(), nn.Conv2d(128, 128, 3, padding=1), nn.ReLU())
        self.d1 = nn.Sequential(nn.Conv2d(128 + 64, 64, 3, padding=1), nn.ReLU(), nn.Conv2d(64, 64, 3, padding=1), nn.ReLU())
        self.out = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        x1 = self.e1(x)
        p1 = self.pool(x1)
        x2 = self.e2(p1)
        p2 = self.pool(x2)
        b = self.b(p2)
        u2 = self.up(b)
        u2 = torch.cat([u2, x2], dim=1)
        x3 = self.d2(u2)
        u1 = self.up(x3)
        u1 = torch.cat([u1, x1], dim=1)
        x4 = self.d1(u1)
        return torch.sigmoid(self.out(x4))

# Load Weights
locator_model = SimpleUNet().to(DEVICE)
try:
    locator_model.load_state_dict(torch.load("locator_unet.pth", map_location=DEVICE))
    locator_model.eval()
    print("✅ Locator U-Net loaded successfully.")
except FileNotFoundError:
    print("❌ Error: 'locator_unet.pth' not found. Did you run the previous cell?")

# --- 3. PROCESSING FUNCTIONS (FIXED) ---
def get_brain_mask(image_pil):
    """
    Runs the U-Net inference on a single image.
    """
    transform = transforms.Compose([
        transforms.Resize(TARGET_SIZE),
        transforms.ToTensor()
    ])
    img_t = transform(image_pil).unsqueeze(0).to(DEVICE)
    
    with torch.no_grad():
        mask_t = locator_model(img_t)
        
    # Convert to Numpy Mask (Binary 0 or 1)
    mask = mask_t.cpu().squeeze().numpy()
    mask = (mask > 0.5).astype(np.uint8) # Threshold at 0.5
    return mask

def apply_hard_attention_and_clahe(image_path):
    # 1. Open Image
    try:
        img_pil = Image.open(image_path).convert("RGB")
    except:
        return None
        
    # 2. Step 1: Get Mask from U-Net
    mask = get_brain_mask(img_pil) # Returns 1-channel mask (224, 224)
    
    # 3. Resize Original to 224x224
    img_cv = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    img_resized = cv2.resize(img_cv, TARGET_SIZE)
    
    # 4. Step 2: Hard Attention (Multiply)
    # FIX: pass the 1-channel 'mask' directly to the mask parameter
    masked_img = cv2.bitwise_and(img_resized, img_resized, mask=mask)
    
    # 5. Step 3: Enhance Contrast (CLAHE)
    lab = cv2.cvtColor(masked_img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    
    final = cv2.normalize(final, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    
    return final

# --- 4. EXECUTE PROCESSING LOOP ---
def generate_hard_attention_dataset():
    if os.path.exists(PROCESSED_DATA_DIR):
        shutil.rmtree(PROCESSED_DATA_DIR)
        
    print("🚀 Starting Hard Attention Processing...")
    
    for cls in CLASSES:
        os.makedirs(os.path.join(PROCESSED_DATA_DIR, cls), exist_ok=True)
        src_dir = os.path.join(RAW_DATA_DIR, cls)
        
        if not os.path.exists(src_dir): continue
            
        files = os.listdir(src_dir)
        print(f"Processing {cls}: {len(files)} images...")
        
        for fname in tqdm(files):
            if not fname.lower().endswith(('.jpg', '.png', '.jpeg')): continue
            
            fpath = os.path.join(src_dir, fname)
            
            # Run the Pipeline
            final_img = apply_hard_attention_and_clahe(fpath)
            
            if final_img is not None:
                save_path = os.path.join(PROCESSED_DATA_DIR, cls, fname)
                cv2.imwrite(save_path, final_img)

# RUN IT
generate_hard_attention_dataset()

✅ Locator U-Net loaded successfully.
🚀 Starting Hard Attention Processing...
Processing Hemorrhagic: 372 images...


100%|██████████| 372/372 [00:07<00:00, 48.67it/s]


Processing Ischemic: 201 images...


100%|██████████| 201/201 [00:04<00:00, 49.90it/s]


Processing Tumor: 2590 images...


100%|██████████| 2590/2590 [00:47<00:00, 54.67it/s]
